The dataset can be found here: https://www.kaggle.com/c/ieee-fraud-detection/data

The dataset has 2 tables: transaction table and identity table. We are mostly interested in the transaction table, which has the following columns:

- TransactionDT: timedelta from a given reference datetime (not an actual timestamp)
- TransactionAMT: transaction payment amount in USD
- ProductCD: product code, the product for each transaction, categorical
- card1 - card6: payment card information, such as card type, card category, issue bank, country, etc. categorical.
- addr1 - addr2: address, categorical
- dist1 - dist2: distance
- P_ and (R__) emaildomain: purchaser and recipient email domain. categorical
- C1-C14: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked.
- D1-D15: timedelta, such as days between previous transaction, etc.
- M1-M9: match, such as names on card and address, etc. categorical
- Vxxx: Vesta engineered rich features, including ranking, counting, and other entity relations.


Based on comments in the [competition host's post](https://www.kaggle.com/c/ieee-fraud-detection/discussion/101203), there are some interesting insights:

- TransactionDT first value is 86400, equals to the number of seconds in a day, suggesting that the unit is seconds.
- addr1 and addr2 are both for purchaser. addr1 is the billing region and addr2 is the billing country.
- dist columns are "distances between (not limited) billing address, mailing address, zip code, IP address, phone area, etc."
- "The logic of our labeling is define reported chargeback on the card as fraud transaction (isFraud=1) and transactions posterior to it with either user account, email address or billing address directly linked to these attributes as fraud too. If none of above is reported and found beyond 120 days, then we define as legit transaction (isFraud=0).
However, in real world fraudulent activity might not be reported, e.g. cardholder was unaware, or forgot to report in time and beyond the claim period, etc. In such cases, supposed fraud might be labeled as legit, but we never could know of them. Thus, we think they're unusual cases and negligible portion."

From the info gathered, we start the EDA process.

First, we load the necessary libraries and set some constants, then load the data into dataframes.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

class InputPath:
    IDENTITY = "/kaggle/input/ieee-fraud-detection/train_identity.csv"
    TRANSACTION = "/kaggle/input/ieee-fraud-detection/train_transaction.csv"

card_columns = [f"card{i}" for i in range(1, 7)]
C_columns = [f"C{i}" for i in range(1, 15)]
D_columns = [f"D{i}" for i in range(1, 16)]
M_columns = [f"M{i}" for i in range(1, 10)]
V_columns = [f"V{i}" for i in range(1, 340)]

In [2]:
df = pd.read_csv(InputPath.TRANSACTION)

V columns are engineered rich features, they may be useful in machine learning but it's hard to understand what they are from EDA so we will drop them.

As previously stated, it is unclear what data dist columns hold and distance cannot be reliably correlated with other columns so we will drop them too.

In [3]:
df.drop(columns=V_columns + ["dist1", "dist2"], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590540 entries, 0 to 590539
Data columns (total 53 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   TransactionID   590540 non-null  int64  
 1   isFraud         590540 non-null  int64  
 2   TransactionDT   590540 non-null  int64  
 3   TransactionAmt  590540 non-null  float64
 4   ProductCD       590540 non-null  object 
 5   card1           590540 non-null  int64  
 6   card2           581607 non-null  float64
 7   card3           588975 non-null  float64
 8   card4           588963 non-null  object 
 9   card5           586281 non-null  float64
 10  card6           588969 non-null  object 
 11  addr1           524834 non-null  float64
 12  addr2           524834 non-null  float64
 13  P_emaildomain   496084 non-null  object 
 14  R_emaildomain   137291 non-null  object 
 15  C1              590540 non-null  float64
 16  C2              590540 non-null  float64
 17  C3        

Let's see what the card columns are

In [4]:
with np.printoptions(threshold=10):
    for col in card_columns:
        uniques = df[col].unique()
        print(f"{col}: {df[col].dtype}, {len(uniques)} unique values")
        print(uniques)
        if np.issubdtype(df[col].dtype, np.number):
            print(f"Value range: {np.nanmin(uniques)} - {np.nanmax(uniques)}, " +
                  ("has NaN" if np.any(np.isnan(uniques)) else "doesn't have NaN"))

card1: int64, 13553 unique values
[13926  2755  4663 ... 13166  8767 18038]
Value range: 1000 - 18396, doesn't have NaN
card2: float64, 501 unique values
[ nan 404. 490. ... 223. 557. 312.]
Value range: 100.0 - 600.0, has NaN
card3: float64, 115 unique values
[150. 117. 185. ... 184. 151. 173.]
Value range: 100.0 - 231.0, has NaN
card4: object, 5 unique values
['discover' 'mastercard' 'visa' 'american express' nan]
card5: float64, 120 unique values
[142. 102. 166. ... 151. 175. 160.]
Value range: 100.0 - 237.0, has NaN
card6: object, 5 unique values
['credit' 'debit' nan 'debit or credit' 'charge card']


From the data, card1 is likely an ID, card4 is card network, card6 is card type, card2, card3, card5 holds numerical data with minimum value 100 and are likely either integers or NaN.

We will use (card1, card4, card6) as card UID.

In [49]:
df["card_uid"] = df["card1"].astype(str) + "_" + df["card4"].astype(str) \
    + "_" + df["card6"].astype(str)
print(len(df["card_uid"].unique()))

217850


How many cards are fraud cards? Are there any card that are marked both as fraud and non-fraud?

In [50]:
frauds = df[df["isFraud"] == 1]
fraud_cards = frauds["card_uid"].unique()
non_fraud_cards = df[df["isFraud"] == 0]["card_uid"].unique()
print("Number of fraud cards:", len(fraud_cards))
print("Number of cards that are both fraud and non-fraud:",
    len(set(fraud_cards).intersection(set(non_fraud_cards))))

Number of fraud cards: 8116
Number of cards that are both fraud and non-fraud: 3112


What can be used for nodes and node data?

The dataset doesn't have enough info to create a graph of nodes and connections.